## 케글의 와인 데이터

- https://www.kaggle.com/datasets/zynicide/wine-reviews

In [19]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

## document loader

In [21]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader('./wine_reviews/winemag-data-130k-v2.csv')
docs = loader.load()
for i, d in enumerate(docs[:2]):
    print(i, d)

0 page_content=': 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia' metadata={'source': './wine_reviews/winemag-data-130k-v2.csv', 'row': 0}
1 page_content=': 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Quinta dos Avidagos 2011 Avidagos Red 

In [22]:
len(docs)

129971

# embedding 모델 객체 생성

In [23]:
# embedding 모델 객체 생성
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL)

# Pinecone객체, index객체 생성

In [26]:
from pinecone import Pinecone, ServerlessSpec


# Pinecone 클라이언트를 초기화(객체생성)
pc = Pinecone(api_key=PINECONE_API_KEY)

- index 있으면 삭제

In [28]:
# index_name = PINECONE_INDEX_NAME

# for idx in pc.list_indexes():
#     if idx.name == index_name:
#         pc.delete_index(index_name)

- Pinecone 객체 생성, index 생성

In [29]:

# pinecone에 index list 가져오기
existing_indexes = pc.list_indexes()

# 이름만 추출
index_names = [index['name'] for index in existing_indexes.indexes]
# print(index_names)

# index 이름이 존재 하지 않으면 생성
if PINECONE_INDEX_NAME not in index_names:
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=1536,  # 모델 차원, openapi embeding model을 사용함. 정확하게 일치
        metric="cosine",  # 모델 메트릭, openapi embeding model 에서 사용하는 것 확인
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Index '{PINECONE_INDEX_NAME}' created successfully.")
else:
    print(f"Index '{PINECONE_INDEX_NAME}' already exists.")


Index 'wine-reviews' created successfully.


- 업로딩에 기간이 좀 걸림, 경우에 따라 50분 정도 걸림
- pinecone 월간 쓰기 작업 제한(Write Unit Limit)을 초과하면 더이상 업로딩을 못함

# Split 객체 생성

In [31]:
# split하기
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 (예: 1000자씩 분할)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=100,
    # length_function=tiktoken_len,  # 토큰 기반 길이 측정    
    length_function=len,  # 문자수   
    separators=["\n\n", "\n", " ", ""]
    )

# 문서를 분할
chunks = text_splitter.split_documents(docs)

In [32]:
len(chunks)

129982

# 배치 크기 단위로 저장하기

- langchain_pinecone 으로 벡터 DB에 저장

In [33]:
# vector sotre에 저장(2시간정도 걸림)
from langchain_pinecone import PineconeVectorStore

batch_size = 100  # 한 번에 처리할 문서 수(최대 vector 수 1000개, 2MB 이내)

for i in range(0, len(chunks), batch_size):
    batch_docs = chunks[i:i+batch_size]
    
    if i == 0:
        # 첫 번째 배치로 벡터 스토어 생성
        vector_store = PineconeVectorStore.from_documents(
            batch_docs,            # batch_size 수 만큼의 chunk
            embedding=embeddings,  # 임베딩 벡터로 변환
            index_name=PINECONE_INDEX_NAME,   # index 이름
            namespace=PINECONE_NAMESPACE      
        )
    else:
        # 이후 배치는 기존 벡터 스토어에 추가, # 내부적으로 임베딩 벡터로 변환
        vector_store.add_documents(batch_docs)    
    
    print(f"배치 {i//batch_size + 1} 완료: {len(batch_docs)}개 문서 업로드")

배치 1 완료: 100개 문서 업로드
배치 2 완료: 100개 문서 업로드
배치 3 완료: 100개 문서 업로드
배치 4 완료: 100개 문서 업로드
배치 5 완료: 100개 문서 업로드
배치 6 완료: 100개 문서 업로드
배치 7 완료: 100개 문서 업로드
배치 8 완료: 100개 문서 업로드
배치 9 완료: 100개 문서 업로드
배치 10 완료: 100개 문서 업로드
배치 11 완료: 100개 문서 업로드
배치 12 완료: 100개 문서 업로드
배치 13 완료: 100개 문서 업로드
배치 14 완료: 100개 문서 업로드
배치 15 완료: 100개 문서 업로드
배치 16 완료: 100개 문서 업로드
배치 17 완료: 100개 문서 업로드
배치 18 완료: 100개 문서 업로드
배치 19 완료: 100개 문서 업로드
배치 343 완료: 100개 문서 업로드
배치 344 완료: 100개 문서 업로드
배치 345 완료: 100개 문서 업로드
배치 346 완료: 100개 문서 업로드
배치 347 완료: 100개 문서 업로드
배치 348 완료: 100개 문서 업로드
배치 349 완료: 100개 문서 업로드
배치 350 완료: 100개 문서 업로드
배치 351 완료: 100개 문서 업로드
배치 352 완료: 100개 문서 업로드
배치 353 완료: 100개 문서 업로드
배치 354 완료: 100개 문서 업로드
배치 355 완료: 100개 문서 업로드
배치 356 완료: 100개 문서 업로드
배치 357 완료: 100개 문서 업로드
배치 358 완료: 100개 문서 업로드
배치 359 완료: 100개 문서 업로드
배치 360 완료: 100개 문서 업로드
배치 361 완료: 100개 문서 업로드
배치 362 완료: 100개 문서 업로드
배치 363 완료: 100개 문서 업로드
배치 364 완료: 100개 문서 업로드
배치 365 완료: 100개 문서 업로드
배치 366 완료: 100개 문서 업로드
배치 367 완료: 100개 문서 업로드
배치 368 완료: 100개 